# Notes FTP :

File Transfer Protocol, ou FTP, est un protocole de communication destiné au partage de fichiers sur un réseau TCP/IP. Il permet, depuis un ordinateur, de copier des fichiers vers un autre ordinateur du réseau.
Utilise une connexion TCP.

port 21 pour les commandes

port 20 pour les données 
(marche en ipv4 et ipv6)


## Wireshark : 
filter par ftp

Dans la première trame FTP !
On peut voir que l'@IPsrc = 10.2.12.16 avec le port 21
Et l'@IPdest = 10.2.4.3 avec le port 55576.

Dans la partie LIST de la trame on vois : USER touriste

On vois donc sur nos manipulation que ici tout est afficher en clair


In [34]:
from scapy.all import *

listetrame=rdpcap("ftp-total.pcapng") #ici on importe notre trame

#trame.summary() #ici on l'affiche 
listetrame.summary()


Ether / IP / TCP 10.2.4.3:39016 > 194.254.242.85:imaps A
Ether / IP / TCP 194.254.242.85:imaps > 10.2.4.3:39016 A / Padding
802.3 b0:aa:77:e0:f8:8d > 01:80:c2:00:00:00 / LLC / STP / Padding
Ether / ARP who has 10.2.4.4 says 10.2.4.1 / Padding
Ether / ARP who has 10.2.4.4 says 10.2.4.1 / Padding
Ether / IP / TCP 10.2.4.3:55576 > 10.2.12.16:ftp S
Ether / IP / TCP 10.2.12.16:ftp > 10.2.4.3:55576 SA
Ether / IP / TCP 10.2.4.3:55576 > 10.2.12.16:ftp A
Ether / IP / TCP 10.2.12.16:ftp > 10.2.4.3:55576 PA / Raw
Ether / IP / TCP 10.2.4.3:55576 > 10.2.12.16:ftp A
Ether / IP / TCP 10.2.4.3:39138 > 194.254.242.85:imaps A
Ether / IP / TCP 194.254.242.85:imaps > 10.2.4.3:39138 A / Padding
802.3 b0:aa:77:e0:f8:8d > 01:80:c2:00:00:00 / LLC / STP / Padding
Ether / IP / TCP 66.102.1.109:imaps > 10.2.4.51:58426 R / Padding
Ether / ARP who has 10.2.4.4 says 10.2.4.1 / Padding
Ether / ARP who has 10.2.4.4 says 10.2.4.1 / Padding
b0:aa:77:e0:f8:8d > b0:aa:77:e0:f8:8d (0x9000) / Raw
Ether / ARP who has 10.2.4

On peut voir qu'a la trame 57 le user apparait donc : 

In [35]:
listetrame[56][Raw] #dans la trame 57 (donc a l'emplacement 56) on affiche la valeur 'raw'

<Raw  load='USER touriste\r\n' |>

On peut coir que dans la trame 128 le Pass apparait : 

In [36]:
listetrame[127][Raw]

<Raw  load='PASS 3aboqphie=3qbc!\r\n' |>

# Programme complet du challenge FTP :

In [1]:
from scapy.all import *

listetrame=rdpcap("ftp-total.pcapng") #ici on importe notre trame

for elm in listetrame : #pour tout les element dans la trame
    try :
        if elm['Ether'].type==0x0800: #si l'en tête etherne a pour type suivant l'IPv4
            if elm['TCP'].dport == 21 or elm['TCP'].sport == 21: #si le fort source ou destination correspont à 21 donc ftp
                #elm.show()
                data = elm['Raw'].load.decode('utf-8') #on associe a une variable data toute la trame que l'on décode en utf8 pour trouver le USER et le PASS
                
                if data.find('USER') != -1 : #on vérifie si USER est dans la chaine (-1 = si user est dans data)
                    #ici data est le contenue de la ligne composé de USER et le nom associer
                    user=[] #on va donc creer un tableau pour stoker les utilisateur
                    user = data.split() #on cdecoupe la data trouver
                    print("le nom d'utilsateur est :", user[1]) #et on affiche que ce qui est lier au user
                
                if data.find('PASS') != -1 : #on vérifie si PASS est dans la chaine (-1 = si user est dans data)
                    pwd=[] #on va donc creer un tableau pour stoker les valeur mot de passe
                    pwd=data.split() #on cdecoupe la data trouver
                    print('Le mot de passe est :', pwd[1]) #et on affiche que ce qui est lier au mot de passe
    
    except:
        continue



le nom d'utilsateur est : touriste
Le mot de passe est : 3aboqphie=3qbc!


le 0x800 correspond à la valeur du champs type IPv4 
